In [4]:
import pandas as pd
import os
import xml.etree.ElementTree as ET ## XML parsing
from collections import defaultdict
import numpy as np
from itunesLibrary import library

from itunes_lib_scan import LibScan
import pickle 

import dateparser
        
import music_tag

%load_ext autoreload
%autoreload 2

In [38]:
lib_name = '/Users/antoine/Developer/itunes-lib-dat/iTunes-Library-2024-03-10'
sd_walkman_path = '/Volumes/SD_CARD/MUSIC'
path_to_music_folder = '/Volumes/MasterAudio/Audio/Gros iTunes/'

create_cover_jpg = False
create_album_jpg = False
complete_missing_cover_art = False
convert_to_non_prog = False
force_scan = True
skip_scanning = False


LS = LibScan(path_to_library_file=lib_name,
             path_to_music_folder = path_to_music_folder,
             create_cover_jpg = create_cover_jpg,
             create_album_jpg = create_album_jpg,
             complete_missing_cover_art = complete_missing_cover_art,
             convert_to_non_prog = convert_to_non_prog,
             force_scan = force_scan,
             skip_scanning = skip_scanning)

LS.get_all_playlists_from_a_lib()
LS.get_all_playlists_with_alternative_path(new_root_folder = sd_walkman_path)


TypeError: __init__() got an unexpected keyword argument 'path_to_library_file'

In [2]:


audio_locations = ['/Volumes/MasterAudio/Audio/']
levels = [0]

df = get_lib_as_csv(lib_name = lib_name)

##########################
# EDIT THE 5 LINES BELOW #
##########################

split_lib(lib_name,
            split_type = ['Genre','Album Scoring',],
              maximum_size_feature = 'Album Size' ,
              maximum_bin_size_in_hours = 4,
              maximum_bin_size_in_gb = 10)

split_lib(lib_name,
            split_type = ['Genre','Album Scoring',],
              maximum_size_feature = 'Album Total Time' ,
              maximum_bin_size_in_hours = 10,
              maximum_bin_size_in_gb = 10)




get_pl_by_locations(
                    lib_name = lib_name, 
                    audio_locations = audio_locations, 
                    levels = levels)


NameError: name 'get_lib_as_csv' is not defined

In [ ]:
def assign_album_features_to_df(path_to_lib,
                        lib_version,
                       pl_list =['0','*', '**', '***', '****', '*****']
                       ):
    """_summary_

    Args:
        path_to_lib (_type_): _description_
        lib_version (_type_): _description_
        pl_list (list, optional): _description_. Defaults to ['*', '**', '*** 1', '**** 1', '***** 1'].

    Returns:
        _type_: _description_
    """
    
    df_lib = assign_score_to_df(path_to_lib,
                        lib_version,
                       pl_list =pl_list
                       )

    
    df_lib.loc[:,'Album'] =  df_lib['Album'].apply(lambda x: x[:-1] if x.endswith(' ') else x)
    df_lib.loc[:,'Album Artist'] =  df_lib['Album Artist'].apply(lambda x: str(x)[:-1] if str(x).endswith(' ') else str(x))
    
    
    group_features = ['Album', 
                        'Album Artist', 
                        'Disc Number',
                        'Genre',  
                        'Disc Count'
                        ]

    grouped_df = df_lib.groupby(by=group_features).apply(group_by_cd).reset_index()

    #grouped_df.loc[:,'cumsum_time'] = np.cumsum((grouped_df.loc[:,'Total Time']/1000)/(4*60*60))

    df_lib = df_lib.merge(grouped_df, on=group_features)

    return df_lib,grouped_df